In [ ]:
import transformers
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
import os
import ftfy
import re  
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer
from transformers import AutoModel
import faiss
import numpy as np
import tiktoken
from sentence_transformers import SentenceTransformer
import json
import torch
from tqdm import tqdm


Preparer les donnees (Documentation de Python)

In [22]:


# BASE_URL = "https://docs.python.org/3/"
# INDEX_URL = BASE_URL + "genindex-all.html"

# HEADERS = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
# }

# OUTPUT_FOLDER = "python_docs_cleaned"
# os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# def get_all_doc_links():
#     response = requests.get(INDEX_URL, headers=HEADERS)
#     response.encoding = "utf-8"
#     if response.status_code != 200:
#         print("Failed to fetch index page")
#         return []

#     soup = BeautifulSoup(response.text, "html.parser")

#     links = []
#     for a in soup.select("a"):
#         href = a.get("href", "")
#         if href.startswith("library/") or href.startswith("tutorial/") or href.startswith("reference/"):
#             full_url = BASE_URL + href
#             links.append(full_url)

#     return list(set(links))  

# def clean_text(text):
#     text = ftfy.fix_text(text)  
#     text = re.sub(r"\n{3,}", "\n\n", text)  
#     text = re.sub(r"([^\n])\n([^\n])", r"\1 \2", text)  
#     return text.strip()

# def scrape_page(url):
#     response = requests.get(url, headers=HEADERS)
#     response.encoding = "utf-8"
#     if response.status_code != 200:
#         return None

#     soup = BeautifulSoup(response.text, "html.parser")

#     title = soup.find("h1")
#     title_text = title.get_text(strip=True) if title else "No Title"

#     content = ""
#     for para in soup.select("p, li, pre"):  
#         text = para.get_text(separator=" ", strip=True)
#         text = clean_text(text)  
#         content += text + "\n\n"  

#     return title_text, content.strip()

# def scrape_and_save():
#     doc_links = get_all_doc_links()
#     print(f"Found {len(doc_links)} documentation pages.")

#     for i, url in enumerate(doc_links):
#         print(f"Scraping {i+1}/{len(doc_links)}: {url}")
#         title, content = scrape_page(url)
#         if content:
#             filename = f"{title.replace(' ', '_').replace('/', '_')}.txt"
#             filepath = os.path.join(OUTPUT_FOLDER, filename)

#             with open(filepath, "w", encoding="utf-8") as f:
#                 f.write(f"Title: {title}\nURL: {url}\n\n{content}")

#     print(f"✅ Saved {len(doc_links)} cleaned pages in '{OUTPUT_FOLDER}'.")

# if __name__ == "__main__":
#     scrape_and_save()


Found 11746 documentation pages.
Scraping 1/11746: https://docs.python.org/3/library/posix.html#index-1
Scraping 2/11746: https://docs.python.org/3/library/importlib.html#importlib.abc.Traversable.is_file
Scraping 3/11746: https://docs.python.org/3/library/asyncio-queue.html#asyncio.Queue.join
Scraping 4/11746: https://docs.python.org/3/library/py_compile.html#cmdoption-python-m-py_compile-q
Scraping 5/11746: https://docs.python.org/3/library/exceptions.html#SyntaxError.lineno
Scraping 6/11746: https://docs.python.org/3/library/dis.html#opcode-LOAD_FAST
Scraping 7/11746: https://docs.python.org/3/library/symtable.html#symtable.SymbolTable.lookup
Scraping 8/11746: https://docs.python.org/3/library/contextvars.html#contextvars.Token
Scraping 9/11746: https://docs.python.org/3/library/functions.html#index-9
Scraping 10/11746: https://docs.python.org/3/library/multiprocessing.html#index-0
Scraping 11/11746: https://docs.python.org/3/reference/datamodel.html#index-51
Scraping 12/11746: http

KeyboardInterrupt: 

Les fichiers sont structures, on utilise le Recursive Chunking avec un decompte de Tokens.

In [ ]:
checkpoint = 'sentence-transformers/all-mpnet-base-v2'
model = AutoModel.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


In [ ]:
def token_splitting(text):
    tokens = tokenizer.encode(text)
    return len(tokens)


La taille maximale d'entree pour le modele qu'on va utiliser pour les embeddings est 384 tokens. 

In [ ]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  
    chunk_overlap=50,  
    length_function=token_splitting,
    separators = ["\n\n","\n",".","?","!"," ",""]
)


In [ ]:

DATA_FOLDER = "data"
FAISS_INDEX_PATH = "db/faiss_index.bin"
META_PATH = "db/faiss_metadata.txt"
CHUNKS_PATH = "db/faiss_chunks.json"

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=384)  
    outputs = model(**inputs)
    return torch.mean(outputs.last_hidden_state, axis=1).detach().numpy().squeeze(0).astype(np.float32)  


dimension = model.config.hidden_size
index = faiss.IndexFlatIP(dimension)  
metadata = []
chunks_data = {}  
chunk_id = 0

for filename in tqdm(os.listdir(DATA_FOLDER), desc="Processing Files", unit="file"):
    if filename.endswith(".txt"):
        file_path = os.path.join(DATA_FOLDER, filename)
        
        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()
        
        chunks = splitter.split_text(text) 
        
        for chunk in tqdm(chunks, desc=f"Embedding {filename}", unit="chunk", leave=False):
            embedding = get_embedding(chunk)  
            faiss.normalize_L2(embedding.reshape(1, -1)) 
            index.add(embedding.reshape(1, -1)) 
            metadata.append(f"{chunk_id}\t{filename}\n")  
            chunks_data[chunk_id] = chunk  
            chunk_id += 1  

faiss.write_index(index, FAISS_INDEX_PATH)

with open(META_PATH, "w", encoding="utf-8") as f:
    f.writelines(metadata)

with open(CHUNKS_PATH, "w", encoding="utf-8") as f:
    json.dump(chunks_data, f, ensure_ascii=False, indent=4) 

print("FAISS index, metadata, and chunks saved successfully!")


Processing Files:   0%|                                                                      | 0/281 [00:00<?, ?file/s]
edding 10.Brief_Tour_of_the_Standard_Library¶.txt:   0%|                                  | 0/17 [00:00<?, ?chunk/s]
edding 10.Brief_Tour_of_the_Standard_Library¶.txt:   6%|█▌                        | 1/17 [00:00<00:03,  4.24chunk/s]
edding 10.Brief_Tour_of_the_Standard_Library¶.txt:  12%|███                       | 2/17 [00:00<00:03,  4.79chunk/s]
edding 10.Brief_Tour_of_the_Standard_Library¶.txt:  18%|████▌                     | 3/17 [00:00<00:03,  4.39chunk/s]
edding 10.Brief_Tour_of_the_Standard_Library¶.txt:  24%|██████                    | 4/17 [00:00<00:02,  4.75chunk/s]
edding 10.Brief_Tour_of_the_Standard_Library¶.txt:  29%|███████▋                  | 5/17 [00:01<00:02,  4.29chunk/s]
edding 10.Brief_Tour_of_the_Standard_Library¶.txt:  35%|█████████▏                | 6/17 [00:01<00:02,  4.26chunk/s]
edding 10.Brief_Tour_of_the_Standard_Library¶.txt:  41%|█████

Voici comment re utiliser les Embeddings apres avoir arreter Notebook

In [31]:
index = faiss.read_index(FAISS_INDEX_PATH)

with open(META_PATH, "r", encoding="utf-8") as f:
    metadata = [line.strip().split("\t") for line in f.readlines()] 

with open(CHUNKS_PATH, "r", encoding="utf-8") as f:
    chunks_data = json.load(f) 

print("FAISS index, metadata, and chunks restored successfully!")


FAISS index, metadata, and chunks restored successfully!


In [33]:
def search(query, top_k=5):
    query_embedding = get_embedding(query).reshape(1, -1)
    faiss.normalize_L2(query_embedding) 
    
    distances, indices = index.search(query_embedding, top_k)  
    
    results = []
    for idx, score in zip(indices[0], distances[0]):  
        if str(idx) in chunks_data: 
            chunk_text = chunks_data[str(idx)]
            results.append((chunk_text, score))
        else:
            print(f"⚠️ Warning: FAISS returned invalid index {idx}, skipping.")

    return results


query = "How to read a CSV in Python"
results = search(query)

for i, (chunk, score) in enumerate(results):
    print(f"Result {i+1}: (Score: {score})\n{chunk}\n")


Result 1: (Score: 0.7100214958190918)
The csv module's reader and writer objects read and write sequences.  Programmers can also read and write data in dictionary form using the DictReader and DictWriter classes.

See also

The Python Enhancement Proposal which proposed this addition to Python.

The csv module defines the following functions:

Return a reader object that will process lines from the given csvfile .  A csvfile must be an iterable of strings, each in the reader's defined csv format. A csvfile is most commonly a file-like object or list. If csvfile is a file object, it should be opened with newline='' . [ 1 ] An optional dialect parameter can be given which is used to define a set of parameters specific to a particular CSV dialect.  It may be an instance of a subclass of the Dialect class or one of the strings returned by the list_dialects() function.  The other optional fmtparams keyword arguments can be given to override individual formatting parameters in the current di

On va definir une fonction qui permet de garder uniquement les entrees liees a la programmation python.

In [ ]:
classifier = pipeline('zero-shot-classification')


No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [ ]:
labels = ['Not Python programming','Python programming']


In [ ]:
def filter_input(input):
    results = classifier(input, candidate_labels = labels)
    if results['labels'][0] == 'Not Python programming':
        return 'Please make sure the question is related to Python Programming'
    return input
